# README
This cookbook was created by Joe Constantino on June 25, 2024. Shout out to Patrick Green for doing the initial work to prove how you can query Headless BI from a notebook.

This cookbook can be configured to query a Tableau Published data source in natural language using the Headless BI REST endpoint. It uses langchain libraries and the langchain development framework to generate a headless BI request payload from a natural language input query. It then executes a chain to generate an answer to the user's input query.

More on the motivation for this prototype can be found in:


*   [Demo](https://drive.google.com/drive/u/0/folders/1drsQBnkA2ZQAgP2Okwo19t9Xdl-wuisM) and [explainer deck](https://docs.google.com/presentation/d/1_MoYuf28cKNtASp10aW3RJbBcIXSl8vnnijQXqzWWjI/edit#slide=id.g2e7ddaa96e5_0_220)
*   PRD (still a work in progress)
[link text](https://docs.google.com/document/d/1S3gWP4b_ZzF_a4uMcG-zZ1Uu7B3mph78VyvAza5hxyQ/edit#heading=h.dp8s1hcyd5za)


To get started with this notebook, make a copy and add your keys to the vault. More info on headless BI configuration requirements can be found [here](https://docs.google.com/document/d/1_um4JzBCYYtzAECduSTKmeY-COQSvAZo5ISyobR4LPY/edit) and [here](https://docs.google.com/document/d/1C5uCVhelRBKO9L40Asm1Cr3EbXBkLPINbxEYdVEvhII/edit#heading=h.fsq4357uza2).

In [1]:
%pip install --upgrade --quiet langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.3 MB/s eta 0:00:00


In [ ]:
pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.2 MB/s eta 0:00:00


In [ ]:
pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
#import all required libraries
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
import pandas as pd
import os
import json
import requests
from google.colab import userdata
from google.colab import drive
import re

from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

# you only need to mount your drive if you have stored the headless BI system prompt in your drive
#drive.mount('/content/drive')
os.environ['OPENAI_API_KEY'] = userdata.get('openai')


In [ ]:
#read in the datasource metadata of the datasource that is listed in vault
def read():
    url = userdata.get('readMetadata')
    payload = json.dumps({
        "connection": {
            "tableauServerName": userdata.get('tableauServerName'),
            "siteId": userdata.get('siteId'),
            "datasource": userdata.get('datasource')
        },
    })

    headers = {
    'Credential-Key': userdata.get('patKey'),
    'Credential-value': userdata.get('patToken'),
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        #df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return data
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)
datasource_metadata = read()

In [ ]:
#define the headless BI query template
def send(query):
    url = userdata.get('vdsUrl')
    payload = json.dumps({
        "connection": {
            "tableauServerName": userdata.get('tableauServerName'),
            "siteId": userdata.get('siteId'),
            "datasource": userdata.get('datasource')
        },
        "query": query
    })

    headers = {
    'Credential-Key': userdata.get('patKey'),
    'Credential-value': userdata.get('patToken'),
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return df
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)

In [ ]:
#load and instantiate prompt; link to base system prompt: https://drive.google.com/file/d/1wqmTSKIwDPgpucwLAgalEyumO2egDJJ5/view?usp=drive_link

# Specify the path to the system prompt
path = '/content/drive/My Drive/Colab Notebooks/headless_bi_prompt_schema.json'

# read in system prompt
with open(path, 'r') as json_file:
    headless_bi_prompt = json.load(json_file)

# add datasource metadata of the connected datasource to the system prompt
headless_bi_prompt['data_model'] = datasource_metadata
headless_bi_prompt_string = json.dumps(headless_bi_prompt)


In [ ]:
def get_utterance():
  query = input('what would you like to know about your data? reply stop if you are done.')
  return query


In [ ]:
active_utterance = get_utterance()
while (active_utterance != 'stop'):
  llm = ChatOpenAI(model="gpt-4o")

  active_prompt_template = ChatPromptTemplate.from_messages([
      SystemMessage(content=headless_bi_prompt_string),
      ("user", "{utterance}")])
  #get_utterance = input('what would you like to know about your data? ')
  prompt_value = active_prompt_template.invoke(
      {
          "utterance": active_utterance
      }
  )
  output_parser = StrOutputParser()
  chain = active_prompt_template | llm | output_parser
  output = chain.invoke(active_utterance)

  #print reasoning
  print(output.split('JSON_payload')[0])

  #parse LLM output and query headless BI
  parsed_output = output.split('JSON_payload')[1]
  match = re.search(r'{.*}', parsed_output, re.DOTALL)
  if match:
    json_string = match.group(0)
    payload = json.loads(json_string)
  #print(payload)
  df = send(payload)
  agent = create_pandas_dataframe_agent(
      ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125"),
      df,
      verbose=True,
      agent_type=AgentType.OPENAI_FUNCTIONS,
      allow_dangerous_code=True
  )
  instruct_header = "answer the following query directly by executing the necessary python code. Give a succinct explanation of the steps you took and how you know the answer is correct: "
  agent.invoke(instruct_header + active_utterance)
  active_utterance = get_utterance()

what would you like to know about your data? reply stop if you are done.which quarter from the first year of the data had the most technology profit?
Reasoning: 
To determine which quarter from the first year of the data had the most technology profit, I need to include the following columns in the query:
1. Quarter - This can be derived from the "Order Date" column using the "DATE_QTR" function.
2. Profit - This can be aggregated using the "SUM" function.
3. Category - To filter for the "Technology" category.

Additionally, I need to apply two filters:
1. Filter the data to include only the first year.
2. Filter the data to include only the "Technology" category.




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['QUARTER(Order Date)'] == 1].loc[df[df['QUARTER(Order Date)'] == 1]['SUM(Profit)'].idxmax()]"}`


QUARTER(Order Date)        1.0000
SUM(Profit)            15872.2573
Name: 2, dtype: float64The quarter from the first year of the data 